# Parte 1, Introducción

<!-- Tomar en consideracion que los chicos no han visto nada practico en deep learning. No se les explicaron cosas como optimizers, schedulers, etc asique hay que ser pedagogicos.

* Motivacion, trabajo con tensores, dimensionalidad alta, optimizacion en gpu, utilidad de computo por batch (eficiencia, robustez del modelo, etc)
 

* introducir la api, las operaciones basicas, operaciones de creacion, operaciones inplace, cambiar la forma de los tensores, etc

* mostrar el uso de la gpu con ejemplos (mostrar nvidia-smi), revisar codigo agnostico al dispositivo -->


------------------------------------------------------
En esta auxiliar vamos a introducir pytorch, un framework para hacer deep learning, y también mostrar dos aplicaciones. Esta herramienta va a ser usada de aquí hasta el final del curso, así que es importante que tengan un conocimiento base sobre este framework.

## Me tinca, pero que es PyTorch exactamente?

Como les decia, PyTorch es un framework para hacer deep learning. Las caracteristicas principales de un framework de este tipo es que permite trabajar y realizar operaciones basicas con tensores (abajo explicamos que son y por qué nos interesan), permite usar facilmente y de forma transparente la GPU, si es que existe (mas delante explicamos por que querriamos hacer esto)  y tambien viene con varias utilidades ya implementadas para acelerar el desarrollo de redes neuronales. Por ejemplo, viene con varios modulos de redes neuronales, como capas lineales (como las que vieron en clases), capas recurrentes, capas convulocionales (estas se ven mas adelante), funciones de activacion, funciones de perdida, etc. Finalmente, y quiza lo mas importante del framework, es que viene con un motor de diferenciacion y propagacion de gradientes automatico, es decir, se guarda un registro de las operaciones que se realizan sobre un tensor y luego se puede calcular automaticamente la derivada de un tensor de salida con respecto a los parametros que estuvieron involucrados en su calculo.




## Ya si, pero tranquilo, qué es un tensor?

Un tensor es una estructura matemática para organizar datos. De toda la vida hemos sabido lo que es un "número", en física y álgebra lineal vimos que podemos organizar varios números en una lista ordenada para obtener un "vector", y luego vimos que podíamos organizar más números en una estructura bidimensional, una "matriz", con filas y columnas.

Los tensores son una forma de generalizar esta idea, entonces decimos que un numero wacho es un tensor de 0 dimensiones, un vector es un tensor de 1 dimensión y una matriz es un tensor de 2 dimensiones. Este concepto nos permite generalizar esta forma de organizar los datos a dimensiones mayores. Podemos hablar de un "cubo" (un tensor de 3 dimensiones), que se podría interpretar como varias matrices apiladas, o más generalmente un tensor de N dimensiones donde N es un número cualquiera (entre mas grande N mas difícil de imaginar :D). Los tensores de dimensiones mayores se los pueden imaginar como listas, donde sus elementos son tensores de dimensiones menores. Esto lo pueden ver en la siguiente fotaza:

(source: knoldus)

![visualizacion tensor](https://i.stack.imgur.com/Lv1qU.jpg)


## Buena, me queda claro qué son, ahora por qué me interesan?

Cuando uno trabaja en deep learning, es muy común encontrarse con datos de alta dimensionalidad. En NLP por ejemplo, en el capitulo de embeddings vimos que es útil representar una palabra como un vector que captura información de la palabra, ya sea de su contexto, de los caracteres que contiene, etc. Tomando esto en cuenta, vemos que una lista de palabras (una frase) la podemos representar como una matriz. A estas alturas ya tenemos un tensor de 2 dimensiones, pero ¿qué pasa si por alguna razón queremos operar sobre varias frases a la vez? ¿Qué pasa si tenemos una lista de frases? Bueno, ahora tenemos un "cubo", un tensor de 3 dimensiones, donde la primera dimensiones corresponde a cada frase dentro del conjunto, la segunda a cada palabra dentro de la frase y finalmente la tercera a cada una las posiciones dentro del vector de embeddings. Esto se repite en muchas más áreas, por ejemplo una imagen RGB es un tensor de 3 dimensiones, un video, donde hay una lista de imágenes se puede interpretar como un tensor de 4 dimensiones, y así.


Otra razón por la que el manejo de tensores se vuelve importante es el concepto de _mini-batch_, donde varios ejemplos se procesan a la vez, efectivamente aumentando en uno la dimensión de los tensores usados para los ejemplos.

En clases se vio que la forma en la que se entrenan las redes neuronales es un proceso iterativo, donde en primer lugar se realiza una predicción que es mala, se calcula una función de perdida, para cada parámetro se calcula el gradiente de la _loss_ con respecto a este parámetro y luego se desciende en la dirección de este gradiente para tratar de llevar los parámetros a los valores que minimizan la función de perdida. Si este proceso iterativo se llevara a cabo de a un ejemplo a la vez, el valor de la _loss_ sería muy dependiente del ejemplo concreto que se acaba de observar y podría no ser representativo de la _loss_ general. Esto resulta en actualizaciones ruidosas de los parámetros, porque la _loss_ para el siguiente ejemplo puede ser muy distinta al valor anterior y así es como los parámetros pueden oscilar y tener dificultad para converger.

Acá es donde nos viene a rescatar el concepto de _mini-batch_, donde los ejemplos se pasan por la red en grupos pequeños para que cada conjunto produzca una  _loss_ más representativa. Esto le agrega robustez al modelo y lo ayuda a converger. El tamaño del _mini-batch_ (cantidad de ejemplos que se pasan a la vez) se vuelve un hiper parámetro de la red. Los valores óptimos de tamaño de _mini-batch_ pueden variar mucho, pero los números que yo he visto varían entre 8 y 32, aunque para algunas aplicaciones he visto valores del orden de 1000.


Pueden leer un poco más [acá](https://machinelearningmastery.com/how-to-control-the-speed-and-stability-of-training-neural-networks-with-gradient-descent-batch-size/), la sección introductoria lo explica un poco en más detalle, y cita al libro [Deep Learning](https://www.deeplearningbook.org/), que es muy weno :-)



## Oye, pero esto esta medio enredado, te podrías sacar unos ejemplos?
La API realmente es muy similar a la de Numpy, asique veremos unos pocos ejemplos nomás. La documentación sobre los tensores la pueden pillar [acá](https://pytorch.org/docs/stable/tensors.html) y la documentacion general de la operaciones sobre tensores que ofrece pytorch esta [acá](https://pytorch.org/docs/stable/torch.html).


In [0]:
# !pip install torch # en sus maquinas
import torch

In [9]:
# Creacion a partir de otra estructura
a = [[2,3,4], [4,5,6]]
t = torch.tensor(a)
print("Desde una lista de listas\n", t)
print("\nDimensiones del tensor\n", t.size())
print("\nNumero de dimensiones del tensor\n", t.dim())

Desde una lista de listas
 tensor([[2, 3, 4],
        [4, 5, 6]])

Dimensiones del tensor
 torch.Size([2, 3])

Numero de dimensiones del tensor
 2


In [10]:
# Creacion de un tensor "vacio"
t = torch.empty(2,3)
print("Tensor vacio\n", t)

Tensor vacio
 tensor([[1.4515e-36, 0.0000e+00, 3.3631e-44],
        [0.0000e+00,        nan, 6.4460e-44]])


In [13]:
# Creacion de tensores con puros 1 o puros ceros
t = torch.ones(2,3,4)
# t = torch.zeros(2,3,4,5)
print("Puros unos\n", t) # notar la tercera dimension

Puros unos
 tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])


In [14]:
# Random sampling
t = torch.empty(3, 2).uniform_() # notar operacion in-place
print("Distribucion uniforme\n", t)

t = torch.randn(2, 3)
print("\nDistribucion normal\n", t)

Distribucion uniforme
 tensor([[0.9071, 0.9928],
        [0.3650, 0.6511],
        [0.3911, 0.0297]])

Distribucion normal
 tensor([[-1.1560, -0.8267, -1.9774],
        [-0.9214,  1.8734,  1.1492]])


In [0]:
# Operaciones matematicas
t = torch.ones(3,4)
print("Operaciones con escalares\n", t + 5)

Operaciones con escalares
 tensor([[6., 6., 6., 6.],
        [6., 6., 6., 6.],
        [6., 6., 6., 6.]])


In [0]:
# Operaciones entre tensores
t1 = torch.ones(2, 3)
t2 = torch.ones(2, 3) * 2
print("Operaciones entre tensores\n", t1 + t2)

Operaciones entre tensores
 tensor([[3., 3., 3.],
        [3., 3., 3.]])


In [0]:
# Tambien se pueden hacer operaciones in-place, se modifica el mismo tensor
t = torch.ones(2,3)
t.add_(1)
print("Suma in-place\n", t)

Suma in-place
 tensor([[2., 2., 2.],
        [2., 2., 2.]])


In [16]:
# Hay veces que es util reorganizar los datos de un tensor, o agregar
# dimensiones 

t = torch.arange(16)
print("Dimensiones de partida\n", t.shape)

t = t.view(-1, 8)
print("\nUsamos el metodo .view() y el -1 para que torch infiera dimensiones\n", t.shape)

t = t.flatten() # Aqui tambien se podria usar .view(-1)
print("\nPodemos volver a aplanar el tensor con .flatten()\n", t.shape)

t = t.view(-1, 4).unsqueeze(1) # tambien podria ser .view(-1, 1, 4)
print("\nPodemos agregar dimensiones sin agregar datos con .unsqueeze()\n", t.shape)

t = t.squeeze()
print("\nCon .squeeze() podemos sacar todas las dimensiones de tamanno 1\n", t.shape)

Dimensiones de partida
 torch.Size([16])

Usamos el metodo .view() y el -1 para que torch infiera dimensiones
 torch.Size([2, 8])

Podemos volver a aplanar el tensor con .flatten()
 torch.Size([16])

Podemos agregar dimensiones sin agregar datos con .unsqueeze()
 torch.Size([4, 1, 4])

Con .squeeze() podemos sacar todas las dimensiones de tamanno 1
 torch.Size([4, 4])


## Pocazos ejemplos... ya pero en honor al tiempo, ¿qué era eso que decías de la GPU?

La GPU es la tarjeta de video de los computadores, un chip que esta especialmente diseñado para manipular muchas matrices de píxeles y aplicarles transformaciones (recuerden que un arreglo de matrices es un tensor de 3 dimensiones), para que luego esos píxeles sean enviados a la pantalla para que los podamos ver. La mayoría de las operaciones tensoriales se pueden paralelizar, por lo que la GPU se aprovecha de esta propiedad y es capaz de realizar operaciones sobre una matriz completa en un solo ciclo de reloj (muy muy rápido). Esto puede mejorar el tiempo de computación hasta por un factor de 100 en cierto casos.

Es por esto que las GPUs se usan tanto en deep learning, porque el deep learning esta basado en operaciones básicas sobre tensores, pero en cantidades enormes. Nos estamos aprovechando de años de investigación y desarrollo de chips para que los gamers puedan tener juegos más fluidos, y les estamos dando un uso ~~productivo~~ científico.



## Me estas diciendo que hay un componente de en mi PC que me sirve para jugar Minecraft y además para hacer investigacion??!! ¿Cómo lo uso?

Otra de las gracias de PyTorch es que permite de forma muy transparente para el usuario interactuar con la GPU. Mover tensores desde la CPU (que es donde se crean por default) hacia la GPU se hace en una pura línea, y además es muy simple escribir código "agnóstico" al dispositivo, lo que significa que si el sistema donde se corre el código dispone de GPUs estas se ocupan, pero si no, se ocupa la CPU nomás.

A continuación hay unos ejemplos, y pueden leer más al respecto [acá](https://pytorch.org/docs/stable/notes/cuda.html).

In [1]:
# Primero usemos un comando de shell para obtener informacion de la GPU
# Recuerden cambiar el runtime del colab
!nvidia-smi

Tue Jun  9 18:54:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
# Verificar si cuda esta disponible en el entorno
print("Habemus GPU?", torch.cuda.is_available())
if torch.cuda.is_available(): # Usar esto para codigo agnostico
    print("Cuantas GPUs me regala Google?", torch.cuda.device_count())

Habemus GPU? True
Cuantas GPUs me regala Google? 1


In [5]:
# Mover tensores entre gpu y cpu
t = torch.empty(3, 4)
print(f"Los tensores se instancian en la {t.device} por default")

t = t.cuda() # .cuda() retorna un nuevo tensor en GPU
print(f"Pero se pueden mover al dispositivo {t.device} usando el methodo .cuda()")

t = torch.empty(3, 4).to("cuda") # Tambien se puede usar con "cpu"
print(f"Tambien se pueden llevar a {t.device} usando el metodo .to()")

Los tensores se instancian en la cpu por default
Pero se pueden mover al dispositivo cuda:0 usando el methodo .cuda()
Tambien se pueden llevar a cuda:0 usando el metodo .to()


In [6]:
# Veamos el uso de la gpu
!nvidia-smi

Tue Jun  9 18:56:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    31W /  70W |    787MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
# Ahora creemos un tensor tremendo
t = torch.empty(6000, 1000, 1000, device="cuda", dtype=torch.int8) # Cada elemento pesa 1 byte

# Y veamos cuanta VRAM estamos usando
!nvidia-smi

Tue Jun  9 18:57:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    31W /  70W |   6511MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Oye pero mi GPU no es tan bacán, no tengo tanta VRAM, me voy a echar el ramo u.u

Pucha, las GPUs son caras, pero por suerte San Google se baña en dinero y nos regala tiempo de GPU en Colab.



\begin{equation}
    Google \downarrow
\end{equation}

<center><img src="https://media.giphy.com/media/Xy2PrQq6BIw7u/giphy.gif"></center>



## Oye, a mi no me gusta derivar, que era eso de que PyTorch deriva automáticamente?

Otra gracia mas de PyTorch (un framework muy agraciado) es que puede almacenar el grafo de computación luego de realizar operaciones sobre tensores. Esto, sumado al hecho que por detrás todas las funciones que usa el framework tienen su respectivas derivadas implementadas, permite que se pueda calcular la derivada de un nodo raíz del grafo de computación (tensores de salida) con respecto una hoja (tensores de entrada). Más información con respecto a autograd se puede obetener [acá](https://pytorch.org/docs/stable/notes/autograd.html).



Veamos un ejemplo super simple (como no me quiero complicar la vida, son puras operaciones punto a punto)

\begin{equation}
\begin{split}
out & = mean(x^2 + log(x)) \\
\frac{\partial out}{\partial x_i} & = \sum_j \frac{\partial mean(y)}{\partial y_j} \frac{\partial (x_j^2 + log(x_j))}{\partial x_i} \\
\end{split}
\end{equation}


Recordar que `out` es un número, mientras que `x` es un tensor. Para este ejemplo voy a suponer que `x` es un tensor de una sola dimensión, pero la explicación aplica también para tensores de más dimensiones.

Veamos primero como queda la primera derivada.

\begin{equation}
\frac{\partial mean(y)}{\partial y_j} = \frac{1}{len(y)}\frac{y_1 + \dots + y_j + \dots + y_n}{\partial y_j} = \frac{1}{len(y)}, \forall j
\end{equation}

Si se fijan, el largo de un vector no depende de ningún valor específico, y como el promedio es una suma de los elementos del tensor, la derivada es $\frac{1}{len(y)}$.

La siguiente derivada es más fácil, ya que son solo operaciones punto a punto. Todas las posiciones que no dependen de la posición _i-esima_ se van a cero. La derivada queda como sigue.
\begin{equation}
\begin{split}
\frac{\partial (x_j^2 + log(x_j))}{\partial x_i} & = 
\begin{cases}
2x_i + \frac{1}{x_i} & \text{ if } j = i \\ 
0 & \text{ if } j \neq i
\end{cases} \\
\sum_j \frac{\partial (x_j^2 + log(x_j))}{\partial x_i} & = 2x_i + \frac{1}{x_i}
\end{split}
\end{equation}

Con estas dos derivadas calculadas podemos volver a la ecuación incial y calcular el valor total de la derivada original. Queda de la siguiente forma (más un poco de algebra para que la ecuación quede bonita).

\begin{equation}
\frac{\partial out}{\partial x_i} = \frac{2x_i^2 + 1}{x_ilen(x)}
\end{equation}

Si probamos con el vector $x=[1,2,3,4]$, reemplazando en la formula deberíamos obtener que el gradiente con respecto a $x$ es $[\frac{3}{4},\frac{9}{8},\frac{19}{12},\frac{33}{16}] = [0.75, 1.125, 1.5833, 2.0625]$

Ahora veamos como pytorch lo hace todo automáticamente.

In [8]:
x = torch.arange(1., 5., requires_grad=True) # Se registra en el grafo de computacion
out = torch.mean(x**2 + torch.log(x))
out.backward() # Se usa backpropagation para calcular gradientes

print("Gradiente de out con respecto a x\n", x.grad)


Gradiente de out con respecto a x
 tensor([0.7500, 1.1250, 1.5833, 2.0625])


## Ohh que cosa mas bacán

Una última cosa, la api de PyTorch es gigante así que yo recomiendo que cuando quieran realizar alguna operación sobre un tensor y no pillen fácilmente en la documentación una forma directa de hacerlo, nos pregunten nomás. Preguntas del tipo: "como hago x en pytorch" o "como hago y en pytorch" son perfectamente razonables, no tengan verguenza de preguntar en el foro :D


Hasta acá llega la introducción a PyTorch, cualquier duda extra nos pueden preguntar extensivamente en el foro o por el grupo de telegram.


_Eso es todo amigos_




In [0]:
%%html
<iframe 
    width="560"
    height="315"
    src="https://www.youtube.com/embed/Ga_RwPmx-N0"
    frameborder="0"
    allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture"
    allowfullscreen>
</iframe>

# Parte 2: BOW + Logistic Regression 


In [0]:
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

Cargamos el dataset para las partes que siguen.

In [0]:
train_data = pd.read_csv('constitucion_train80.csv', sep=',')
test_data = pd.read_csv('constitucion_test20.csv', sep=',')

Veamos primero que es lo que tiene el dataset:

In [0]:
train_data.head()

,argument,constitutional_concept
0,"básica, permite ser, existir. comprende derech...",A la vida
1,aceptación de diversas tendencias y opiniones.,Tolerancia
2,-,De protección y conservación de patrimonio his...
3,inculcar la inclusión de todos los sectores de...,Inclusión
4,nuestro deber como ciudadano es cumplir con es...,Cumplimiento de obligaciones fiscales


Creamos el vocabulario que necesitamos para generar el modelo de Bag of Words.

In [0]:
palabras = {}
for sent in [x.split() for x in (list(train_data['argument'])+list(test_data['argument']))]:
    for word in sent:
        if word not in palabras:
            palabras[word] = len(palabras)


VOCAB_SIZE = len(palabras)

Cuantos labels vamos a clasificar?

In [0]:
labels_to_idx = {}
for label in set(list(train_data['constitutional_concept'])+list(test_data['constitutional_concept'])): # Utilizamos 'set' para obtener los valores únicos
      if label not in labels_to_idx:
          labels_to_idx[label] = len(labels_to_idx)

NUM_LABELS = len(labels_to_idx) 

In [0]:
print(VOCAB_SIZE)
print(NUM_LABELS)

48318
161


Tenemos entonces un vocabulario de 70401 palabras y queremos clasificar entre 160 tópicos distintos.

Las siguientes funciones auxiliares nos permiten obtener las representaciones BoW.

In [0]:
def make_bow_vector(batch): # Representacion en bag of words (sentence es una lista de las palabras)
    ret = []
    for sentence in batch:
        vec = torch.zeros(VOCAB_SIZE)
        for word in sentence.split():
            vec[palabras[word]] += 1 # Le sumamos 1 a la columna que representa la palabra
        ret.append(vec.view(1, -1))
    return tuple(ret)


def make_target(batch): # Tensor que contiene one hot spot del label.
    ret = []
    for label in batch:
        ret.append(torch.LongTensor([labels_to_idx[label]]))
    return tuple(ret)

Generamos un "Dataloader" para los datos de training. Este dataloader nos permite separar los datos en mini-batches.

In [0]:
bow_train_data = []
for index, row in train_data.iterrows(): # Iteramos en el df de pandas de entrenamiento
    bow_train_data.append((row['argument'],row['constitutional_concept']))
trainloader = torch.utils.data.DataLoader(bow_train_data, batch_size=16,
                                          shuffle=True, num_workers=10)
dataiter = iter(trainloader)

Podemos ver que nos entrega una iteracion del dataloader.

In [0]:
argument, concept = dataiter.next()
print(argument)
print(concept)

('asegurar que nuestros derechos sean respetados y considerados', 'es el deber mínimo de todo ciudadano.', 'asegurar la prolijidad en todos los aspectos del estado.', 'más descentralización y poder político y económico a las regiones.', 'conservar historia y patrimonio arquitectónico, las expresiones culturales, monumentos históricos y patrimonio arqueológicoconservar y respetar culturas y lenguas extintas y las vivas.fundamental para tener una nación con identidad o identidades para crear un sentido de pertenencia.', 'todos los ciudadanos deberían ser iguales ante la ley sin importar sus diferencias.', 'en base a la situación actual que vive el país se hace necesario un mejoramiento en los mecanismos de fiscalización.', 'para superar el manejo o manipulaciòn de la opinión publica donde se esconden datos, se habla mal de algunas fuerzas polìticas, sin darles oportunidad de que ellos den su propia versiòn.', 'la constitucion debe contemplar como se estructura el gobierno las fucniones q

Generamos también el conjunto de testing:

In [0]:
bow_test_data = []
for index, row in test_data.iterrows(): # Iteramos en el df de pandas de entrenamiento
    bow_test_data.append((row['argument'],row['constitutional_concept']))
testloader = torch.utils.data.DataLoader(bow_test_data, batch_size=16,
                                          shuffle=True, num_workers=10)

Configuramos nuestro clasificador, extendiendo la clase nn.Module de pytorch.

In [0]:
class BoWClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()  # Primero llamamos al constructor de la superclase.
        # Luego le agregamos los parametros necesarios para el modelo, en este caso pasamos de un tensor de largo vocab_size a otro de largo num_labels.
        self.linear = nn.Linear(vocab_size, num_labels)

    def forward(self, bow_vec):
        # Pasamos el input por la capa lineal y terminamos con su vector de labels.
        return self.linear(bow_vec)

Inicializamos el modelo, entregando que funcion de loss y que optimizador vamos a usar.

In [0]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE).cuda()
loss_function = nn.CrossEntropyLoss() # CrossEntropy calcula el softmax, si no seria necesario agregarla al modelo
optimizer = optim.SGD(model.parameters(), lr=0.1)

Sólo para demostrar que sí se logró entrenar algo, veremos el accuracy pasandolo por el modelo antes de entrenar

In [0]:
total = 0
good = 0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    total += len(inputs)
    bow_vec = torch.stack(make_bow_vector(inputs)).cuda()
    target = torch.stack(make_target(labels)).cuda()
    outputs = model(bow_vec)
    equal_classes = (torch.argmax(outputs, dim=2) == target).sum() # Suma todas las clases que son iguales
    good += equal_classes.item() # Extraemos el valor del tensor
print('Accuracy: {0}'.format(good/total))

Accuracy: 0.006866827379647893


Y el paso mas importante: Entrenar!

In [0]:
for epoch in range(15):
    running_loss = 0.0
    t = tqdm(enumerate(trainloader, 0), leave=True) # Barra que muestra progreso
    for i, data in t:
        inputs, labels = data
        model.zero_grad() # Limpiar los gradientes antes de cada iteración

        bow_vec = torch.stack(make_bow_vector(inputs)).cuda()
        target = torch.stack(make_target(labels)).cuda()
        # Hacer un forward pass y ver el resultado
        outputs = model(bow_vec)
        # Calcular la loss y los gradientes

        loss = loss_function(outputs.squeeze(1), target.squeeze(1))
        loss.backward()

        # Actualizar los parametros del modelo
        optimizer.step()
        
        running_loss += loss.item()
        if i % 50 == 0:    # print cada 50 mini-batches
            # La loss esta dividida por 50 ya que se fue acumulando durante 50 mini-batches
            t.set_description("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss/50)) 
            t.refresh() # Actualizar la barrita
            running_loss = 0.0 # Reseteamos la loss

[1,  3501] loss: 174.588: : 3523it [00:23, 150.91it/s]
[2,  3501] loss: 158.775: : 3523it [00:24, 146.26it/s]
[3,  3501] loss: 144.356: : 3523it [00:24, 146.48it/s]
[4,  3501] loss: 136.225: : 3523it [00:23, 149.77it/s]
[5,  3501] loss: 126.767: : 3523it [00:23, 151.57it/s]
[6,  3501] loss: 120.383: : 3523it [00:23, 152.60it/s]
[7,  3501] loss: 123.153: : 3523it [00:23, 152.82it/s]
[8,  3501] loss: 116.175: : 3523it [00:22, 153.37it/s]
[9,  3501] loss: 113.481: : 3523it [00:23, 151.97it/s]
[10,  3501] loss: 111.522: : 3523it [00:23, 152.70it/s]
[11,  3501] loss: 103.419: : 3523it [00:23, 151.81it/s]
[12,  3501] loss: 96.769: : 3523it [00:22, 153.62it/s]
[13,  3501] loss: 101.566: : 3523it [00:23, 151.36it/s]
[14,  3501] loss: 105.740: : 3523it [00:23, 152.40it/s]
[15,  3501] loss: 104.850: : 3523it [00:23, 152.36it/s]


Accuracy post entrenamiento:

In [0]:
total = 0
good = 0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    total += len(inputs)
    bow_vec = torch.stack(make_bow_vector(inputs)).cuda()
    target = torch.stack(make_target(labels)).cuda()
    outputs = model(bow_vec)
    equal_classes = (torch.argmax(outputs, dim=2) == target).sum()
    good += equal_classes.item()
print('Accuracy: {0}'.format(good/total))

Accuracy: 0.4546716341588136


Para ser un problema con 160 clases, esta accuracy no es mala. Un clasificador que tira una clase al azar dentro de las 160 tendría 0.00625 de accuracy.

Parametros con los que pueden jugar:
- Cantidad de epochs
- Batch_size
- Learning rate del optimizer
- Cambiar la funcion de loss

# Parte 3, embeddings + feed forward

Acá viene otro ejemplo más, esta vez usando una capa de embeddings y una red feed forward. Para este ejemplo me inspire ~~totalmente~~ parcialmente de [este tutorial](https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html) de PyTorch.

La próxima auxiliar veremos más utilidades de torchtext, vamos a probar usando embeddings preentrenados y vamos a ver ejemplos de otras arquitecturas más complejas.

In [0]:
%%capture --no-stderr
!pip install --upgrade torchtext

In [13]:
import csv
import http.client
import gzip

from itertools import islice
from random import sample

from tqdm import tqdm

# Primero descarguemos el dataset
HOST = "raw.githubusercontent.com"
URL = "/uchile-nlp/ArgumentMining2017/master/data/complete_data.csv.gz"

# Descargar los datos directamente de github
conn = http.client.HTTPSConnection(HOST)
conn.request("GET", URL)

# Para este ejemplo solo voy a trabajar con documentos de la categoria 1, "Valores"
dataset = tuple(
    # Usemos lowercase para que el vocabulario no quede tan grande
    (row["constitutional_concept"], row["argument"].lower())
    for row in tqdm(
            csv.DictReader(
                gzip.open(conn.getresponse(), mode="rt"),
                strict=True,
                escapechar="\\",
            )
        )
    # Usamos solo el primer topico, y hay algunos argumentos vacios
    if row["topic"] == "1" and row["argument"]
)

# Mostremos algunos ejemplos
for example in sample(dataset, 3):
    print("\nEjemplo aleatorio:\n", example)

205357it [00:01, 138145.85it/s]


Ejemplo aleatorio:
 ('Bien Común / Comunidad', 'se requiere trabajar en base a intereses colectivos y no privilegiando intereses personales.')

Ejemplo aleatorio:
 ('Seguridad', '-leyes demasiado blandas y flexibles. -formular leyes más duras. -crear cárceles "reformatorias".')

Ejemplo aleatorio:
 ('Respeto / Conservación de la naturaleza o medio ambiente', 'para los habitantes de un sector rural es muy importante el cuidado del medio ambiente, va en directo beneficio de la localidad.  cuidado del agua y de los alimentos')


In [0]:
%%capture --no-stderr
# Ahora armemos un vocabulario, para esto necesitamos un tokenizador,
# pero torchtext no viene con un tokenizador para espannol asi que
# hay que bajar uno de spacy
!python -m spacy download es

In [16]:
# Ahora si armemos el vocabulario y la lista de labels
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("spacy", "es")
vocab = build_vocab_from_iterator(
    tokenizer(argument) for _, argument in dataset
)
labels = list({doc[0] for doc in dataset})
label_map = {label: index for index, label in enumerate(labels)}

print("\nTamanno del vocabulario:", len(vocab))
print("Algunas palabras del vocabulario:", sample(vocab.itos, 5))
print("\nCantidad de labels:", len(labels))
print("Algunos labels:", sample(labels, 3))

53780lines [00:04, 12948.51lines/s]


Tamanno del vocabulario: 23765
Algunas palabras del vocabulario: ['requeridas', 'incansablemente', 'gretuita', 'soberanos', 'apropiarnos']

Cantidad de labels: 55
Algunos labels: ['Justicia social', 'Dignidad', 'Equidad']


In [17]:
# Ahora con este vocabulario podemos armar un set de train y uno de validacion
import torch
from torch.utils.data.dataset import random_split
train_len = int(len(dataset) * 0.8)

train_dataset, validation_dataset = [
    [
        (
            label_map[item[0]],
            torch.tensor([vocab[token] for token in tokenizer(item[1])]),
        )
        for item in split
    ]
    for split in random_split(dataset, [train_len, len(dataset) - train_len])
]

print("Algunos ejemplos del dataset:")
for example in sample(train_dataset, 3):
    print(example)

Algunos ejemplos del dataset:
(16, tensor([ 201,   12,  357,  413, 1845,   35,  185,   21,  590,  812,    2,    5,
          32,    4,   38, 3999,    3]))
(0, tensor([  50,   45,    4,  208,   10,   52,  113,   33,    7,  261,  558,    3,
        1473,    9,   91,   36,   68,  746,    4,    5,   40,   16,  151,    6,
         201,    3]))
(16, tensor([3085]))


In [0]:
# Pum ahora hagamos la arquitectura
# simplecita, un capa de embedding, y luego una red feed forward de 
import torch.nn as nn
import torch.nn.functional as F

# Red neuronal con una sola capa escondida
class ArgumentClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, hidden_size, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, pad_idx)
        self.fc1 = nn.Linear(embed_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_class)

    def forward(self, batch):
        # (B, N, E) -> (B, E)
        # La representacion de un documento sera el promedio de los
        # embeddings de sus palabras.
        z = self.embedding(batch).mean(dim=1)
        z = F.relu(self.fc1(z))
        z = F.relu(self.fc2(z))
        return torch.softmax(z, -1)

In [0]:
# Para mas adelante, necesitamos definir una funcion que determine
# como convertir un conjunto de items de nuestro dataset en un batch,
# recordando que los tensores en pytorch tienen que ser homogeneos

# Esta funcion recibe una lista de muestras del dataset y debe retornar
# tensores que agrupan estas muestras. Si cada ejemplo de nuestro dataset
# contiene 2 elementos y nuestro tamanno de batch es de 16, entonces esta funcion
# debe retorna una tupla de 2 tensores, cada uno de dimension 16 x ... 
from itertools import zip_longest

def generate_batch(batch):
    return (
        # En este caso como los labels son numeros, el tensor es de 1 dimension
        # de tamanno batch_size
        torch.tensor([item[0] for item in batch]),

        # En este caso se retorna un tensor de 2 dimensiones, batch_size x N,
        # donde N es mayor largo de los ejemplo en el batch. Aca se realiza
        # padding de los ejemplos mas cortos.
        torch.tensor(
            list(
                zip(
                    *zip_longest(
                        *[item[1] for item in batch], fillvalue=vocab["<pad>"]
                    )
                )
            )
        ),
    )


In [0]:
# Ahora creamos funciones para entrenar y validar el modelo
from torch.utils.data import DataLoader


def train_func(train_dataset):

    # Entranamos el modelo
    train_loss = 0
    train_acc = 0
    data = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=generate_batch,
    )
    for i, (cls, text) in enumerate(data):
        optimizer.zero_grad()
        cls, text = cls.to(device), text.to(device)
        output = model(text)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Ajustar el learning rate
    scheduler.step()

    return train_loss / len(train_dataset), train_acc / len(train_dataset)


def test(test_dataset):
    test_loss = 0
    acc = 0
    data = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, collate_fn=generate_batch
    )
    for cls, text in data:
        cls, text = cls.to(device), text.to(device)
        with torch.no_grad():
            output = model(text)
            loss = criterion(output, cls)
            test_loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return test_loss / len(test_dataset), acc / len(test_dataset)

In [28]:
# Ahora por fin tenemos todo lo necesario para entrenar el modelo.
import time

N_EPOCHS = 5
LEARN_RATE = 4.0
STEP_SIZE = 1
BATCH_SIZE = 32
EMBED_DIM = 100
HIDDEN_SIZE = 200

device = "cuda" if torch.cuda.is_available() else "cpu"

model = ArgumentClassifier(
    vocab_size=len(vocab),
    embed_dim=EMBED_DIM,
    num_class=len(labels),
    hidden_size=HIDDEN_SIZE,
    pad_idx=vocab["<pad>"],
).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, STEP_SIZE)


for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(train_dataset)
    valid_loss, valid_acc = test(validation_dataset)

    secs = int(time.time() - start_time)
    mins = secs // 60
    secs = secs % 60

    print(
        f"Epoch: {epoch + 1}", f" | time in {mins} minutes, {secs} seconds",
    )
    print(
        f"\tLoss: {train_loss:.4f}(train)\t|"
        f"\tAcc: {train_acc * 100:.1f}%(train)"
    )
    print(
        f"\tLoss: {valid_loss:.4f}(valid)\t|"
        f"\tAcc: {valid_acc * 100:.1f}%(valid)"
    )


Epoch: 1  | time in 0 minutes, 5 seconds
	Loss: 0.1218(train)	|	Acc: 14.7%(train)
	Loss: 0.1208(valid)	|	Acc: 18.2%(valid)
Epoch: 2  | time in 0 minutes, 5 seconds
	Loss: 0.1203(train)	|	Acc: 19.0%(train)
	Loss: 0.1206(valid)	|	Acc: 18.9%(valid)
Epoch: 3  | time in 0 minutes, 5 seconds
	Loss: 0.1202(train)	|	Acc: 19.2%(train)
	Loss: 0.1206(valid)	|	Acc: 19.0%(valid)
Epoch: 4  | time in 0 minutes, 5 seconds
	Loss: 0.1202(train)	|	Acc: 19.2%(train)
	Loss: 0.1206(valid)	|	Acc: 19.1%(valid)
Epoch: 5  | time in 0 minutes, 5 seconds
	Loss: 0.1202(train)	|	Acc: 19.3%(train)
	Loss: 0.1206(valid)	|	Acc: 19.1%(valid)
